In [1]:
import torch as t
import torch.nn as nn
import math

In [13]:
Bilstm = nn.LSTM(input_size=1, hidden_size=2, num_layers=1, batch_first=True, bidirectional=True)

In [10]:
input = t.randn(5, 1, 4)
input

tensor([[[ 1.1568,  0.3701,  1.2371, -1.2127]],

        [[-0.4818,  1.3980, -0.1983, -0.2674]],

        [[ 0.7368,  0.9752, -1.3589, -0.1075]],

        [[ 0.1641,  1.0206, -2.4703,  1.4388]],

        [[-0.5479, -0.2959, -2.2385, -0.8678]]])

In [11]:
input = input.permute(0, 2, 1)

In [15]:
result = Bilstm(input)

In [17]:
len(result)

2

In [18]:
result[0].size()

torch.Size([5, 4, 4])

In [19]:
input.size()

torch.Size([5, 4, 1])

In [24]:
result[1][1].size()

torch.Size([2, 5, 2])

In [27]:
fc = nn.Linear(in_features=4 * 2 * 2, out_features=1)

In [28]:
fc_input = result[0].reshape(5, -1)

In [30]:
fc(fc_input)

tensor([[0.2010],
        [0.2005],
        [0.2059],
        [0.1561],
        [0.1130]], grad_fn=<AddmmBackward>)

In [39]:
result[0].reshape(result[0].shape[0], -1)

tensor([[ 0.0671, -0.0777, -0.1432, -0.2131,  0.0172, -0.0949, -0.1220, -0.1665,
          0.0782, -0.1187, -0.1270, -0.0792, -0.0588, -0.0093, -0.0569,  0.0400],
        [-0.0618,  0.0146, -0.1132, -0.1894,  0.0623, -0.0758, -0.1470, -0.1851,
         -0.0274, -0.0638, -0.1070, -0.1327, -0.0530, -0.0502, -0.0699, -0.0761],
        [ 0.0298, -0.0660, -0.1382, -0.1866,  0.0606, -0.1044, -0.1467, -0.1097,
         -0.0648,  0.0188, -0.0840,  0.0048, -0.0666, -0.0084, -0.0721, -0.0870],
        [-0.0204, -0.0370, -0.1234, -0.1055,  0.0489, -0.0915, -0.1412,  0.0257,
         -0.0686,  0.1505, -0.0585,  0.1970,  0.0506, -0.0341, -0.0896, -0.0952],
        [-0.0648,  0.0209, -0.1230, -0.0622, -0.0735,  0.0094, -0.1204,  0.0281,
         -0.1007,  0.2084, -0.0657,  0.2011, -0.1361,  0.1833, -0.0616, -0.0120]],
       grad_fn=<UnsafeViewBackward>)

In [38]:
result[0].shape[0]

5

In [52]:
class Bicut(nn.Module):
    def __init__(self, seq_len=300, input_size=1, lstm_hiden_size=128, lstm_layers=2, lstm_dropout=0.4, fc_dimensions=256):
        super(Bicut, self).__init__()
        self.Bilstm = nn.LSTM(input_size=input_size, hidden_size=lstm_hiden_size, num_layers=lstm_layers,
                              batch_first=True, dropout=lstm_dropout, bidirectional=True)
        self.FC = nn.Linear(in_features=seq_len *lstm_hiden_size * 2, out_features=fc_dimensions)
        self.Softmax = nn.Sequential(
            nn.Linear(in_features=fc_dimensions, out_features=2),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        x = self.Bilstm(x)[0]
        x = x.reshape(x.shape[0], -1)
        x = t.sigmoid(self.FC(x))
        return self.Softmax(x)

In [53]:
model = Bicut()
input = t.randn(194, 300, 1)
input.shape

torch.Size([194, 300, 1])

In [54]:
result = model.forward(input)
result.shape

torch.Size([194, 2])

In [56]:
test_i = t.randn(5, 2, 16)
fc(test_i).size()


torch.Size([5, 2, 1])